In [2]:
import pandas as pd
import numpy as np
import pandas as pd
# # параметри виведення 
pd.set_option("display.max_columns", 500) # кількість колонок
pd.set_option("display.max_rows", 1000) # кількість рядків
pd.set_option("display.max_colwidth", 300) # ширина колонок
pd.set_option("display.precision", 7) # кількість знаків після коми

# # вимикаємо зайві попередження
import warnings
warnings.filterwarnings("ignore") 

# # друк всіх результатів в одній комірці а не тільки останнього
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

df= pd.read_csv('survey_results_public.csv')
shema = pd.read_csv('survey_results_schema.csv')

In [3]:
#Скільки респондентів пройшли опитування?
response_count = df.count()['ResponseId']
print(f'Респонденти що пройшли опитування: {response_count}')

Респонденти що пройшли опитування: 89184


In [4]:
#Скільки респондентів відповіли на всі запитання?
questions = set(shema.qname.unique()) & set(df.columns)
all_quesions = df.dropna(subset=questions).shape[0]
print(f"Респонденти що відповіли на всі питання: {all_quesions}")

Респонденти що відповіли на всі питання: 2032


In [6]:
#Які значення мір центральної тенденції для досвіду (WorkExp) респондентів?
df_mean = df['WorkExp'].mean()
df_median = df['WorkExp'].median()
df_mode = df['WorkExp'].mode()[0]
df_mean_round = round(df_mean,2)
print(f'Середній арифметичний досвід роботи:{df_mean_round}, медіанна: {df_median}, та мода: {df_mode}')

Середній арифметичний досвід роботи:11.41, медіанна: 9.0, та мода: 5.0


In [7]:
#Скільки респондентів працює віддалено?
df_rw = df['RemoteWork'] == 'Remote'
count_df_rw = df_rw.sum()
print(f'Респонденти що працюють лише віддалено: {count_df_rw}')

Респонденти що працюють лише віддалено: 30566


In [8]:
#Який відсоток респондентів програмує на Python?
df_python_code = df['LanguageHaveWorkedWith'].str.contains('Python',case=False, na=False)
count_df_python_code = df_python_code.sum()
persent_count_df_python_code = count_df_python_code / response_count*100
round_count_df_python_code = round(persent_count_df_python_code,2)
print(f'Відсоток респондентів що програмують на Python: {round_count_df_python_code}%')

Відсоток респондентів що програмують на Python: 48.39%


In [9]:
#Скільки респондентів навчалося програмувати за допомогою онлайн курсів?
df_learn_code_courses = df['LearnCode'].str.contains('Online Courses or Certification',case=False, na=False)
coubt_df_learn_code_courses = df_learn_code_courses.sum()
print(f'Респонденти що вчилися програмуванню на онлайн курсах: {coubt_df_learn_code_courses}')

Респонденти що вчилися програмуванню на онлайн курсах: 43201


In [13]:
#Серед респондентів що програмують на Python в групуванні по країнам, яка середня та медіанна сума компенсації (ConvertedCompYearly) в кожній країні?
df_python_programmers = df[df['LanguageHaveWorkedWith'].str.contains('Python', case=False, na=False)]
grouped_data = df_python_programmers.groupby('Country')['ConvertedCompYearly'].agg(['mean', 'median'])
grouped_data.head(5)

,mean,median
Country,,
Afghanistan,665.0000000,48.0
Albania,28008.6000000,11844.0
Algeria,8336.3333333,6586.0
Andorra,32127.0000000,32127.0
Angola,662.0000000,662.0


In [14]:
#Які рівні освіти мають 5 респондентів з найбільшою компенсацією?
df_top_5 = df.sort_values(by='ConvertedCompYearly', ascending = False).head(5)
degree_top_5 = df_top_5[['EdLevel','ConvertedCompYearly']]
print(degree_top_5)

                                              EdLevel  ConvertedCompYearly
53268  Professional degree (JD, MD, Ph.D, Ed.D, etc.)           74351432.0
77848  Professional degree (JD, MD, Ph.D, Ed.D, etc.)           73607918.0
66223    Bachelor’s degree (B.A., B.S., B.Eng., etc.)           72714292.0
28121                       Primary/elementary school           57513831.0
19679  Professional degree (JD, MD, Ph.D, Ed.D, etc.)           36573181.0


In [15]:
#В кожній віковій категорії, який відсоток респондентів програмує на Python?
df_python_coders = df[df['LanguageHaveWorkedWith'].str.contains('Python', case=False, na=False)]
grouped_age = (df_python_coders.groupby('Age')['Age'].count() / df.groupby('Age')['Age'].count()) * 100
print(grouped_age)

Age
18-24 years old       61.3908873
25-34 years old       47.5772250
35-44 years old       41.4426261
45-54 years old       38.4569234
55-64 years old       36.4976415
65 years or older     30.9137489
Prefer not to say     41.2026726
Under 18 years old    68.6288760
Name: Age, dtype: float64


In [16]:
#Серед респондентів що знаходяться в 75 перцентилі за компенсацією середнього і працюють віддалено, які індустрії є найрозповсюдженішими?
percentile_75 = df['ConvertedCompYearly'].quantile(0.75) 
respondent_high_level_remote = df[(df['ConvertedCompYearly'] >= percentile_75) & (df['RemoteWork'] == 'Remote')]
industry_counts = respondent_high_level_remote['DevType'].explode().value_counts()
industry_counts.head(5)

DevType
Developer, full-stack      2362
Developer, back-end        1512
Developer, front-end        375
Engineering manager         312
Other (please specify):     273
Name: count, dtype: int64